## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-16-16-56-33 +0000,nyt,Susie Wiles Acknowledges Trump’s ‘Score Settli...,https://www.nytimes.com/2025/12/16/us/politics...
1,2025-12-16-16-54-51 +0000,nyt,New York Accuses UPS of Seasonal Worker Wage T...,https://www.nytimes.com/2025/12/16/business/up...
2,2025-12-16-16-49-24 +0000,nyt,Europe May Roll Back Combustion Engine Ban,https://www.nytimes.com/2025/12/16/business/ec...
3,2025-12-16-16-49-23 +0000,cbc,"Damaged homes collapsing in Gaza, trapping fam...",https://www.cbc.ca/news/world/winter-storm-aft...
4,2025-12-16-16-47-36 +0000,nyt,Zelensky Expresses Wary Optimism About Russia-...,https://www.nytimes.com/2025/12/16/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2397/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
84,bondi,41
3,trump,41
85,beach,30
42,reiner,24
41,rob,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
336,2025-12-15-18-30-23 +0000,nypost,Officer learned he was going to be a dad days ...,https://nypost.com/2025/12/15/world-news/shot-...,156
348,2025-12-15-17-48-41 +0000,nypost,Bondi Beach father-son terror suspects travele...,https://nypost.com/2025/12/15/world-news/bondi...,153
281,2025-12-15-21-15-00 +0000,wsj,Democrats and some Republicans say President T...,https://www.wsj.com/politics/policy/trump-rob-...,131
289,2025-12-15-20-58-52 +0000,latimes,Chabria: Trump's callous political attack on R...,https://www.latimes.com/california/story/2025-...,119
54,2025-12-16-14-39-25 +0000,wapo,"Rabbi, photographer and 10-year-old are among ...",https://www.washingtonpost.com/world/2025/12/1...,115


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
336,156,2025-12-15-18-30-23 +0000,nypost,Officer learned he was going to be a dad days ...,https://nypost.com/2025/12/15/world-news/shot-...
281,123,2025-12-15-21-15-00 +0000,wsj,Democrats and some Republicans say President T...,https://www.wsj.com/politics/policy/trump-rob-...
249,71,2025-12-15-22-51-00 +0000,wsj,Police Ask Public to Help Identify Potential B...,https://www.wsj.com/us-news/brown-university-s...
304,52,2025-12-15-20-13-27 +0000,nypost,US close to strongest ever Ukraine security gu...,https://nypost.com/2025/12/15/world-news/us-cl...
262,49,2025-12-15-22-00-24 +0000,bbc,Epstein's UK flights had alleged British abuse...,https://www.bbc.com/news/articles/cy4709kylkxo...
232,45,2025-12-15-23-42-07 +0000,nypost,‘Anti-capitalist’ New Year’s Eve LA terror plo...,https://nypost.com/2025/12/15/us-news/anti-cap...
121,43,2025-12-16-11-07-53 +0000,bbc,Dashcam video shows couple trying to stop gunm...,https://www.bbc.com/news/articles/cx2341yx719o...
79,39,2025-12-16-13-33-47 +0000,nypost,White House chief of staff Susie Wiles: AG Pam...,https://nypost.com/2025/12/16/us-news/ag-pam-b...
348,39,2025-12-15-17-48-41 +0000,nypost,Bondi Beach father-son terror suspects travele...,https://nypost.com/2025/12/15/world-news/bondi...
241,36,2025-12-15-23-10-15 +0000,nyt,Ford Will Take $19.5 Billion Hit as It Rolls B...,https://www.nytimes.com/2025/12/15/business/fo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
